Select a drug from the dataset. Imagine that a leader for that product has asked you the following question: “What aspects do patients like about using our product, and what do people dislike?”
Use the data to write a report for this leader that will address their question.